In [ ]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

# Download files

Downlaods all PDF files from a url

In [ ]:
#Create folder, if one doesn't exist
folder_location = '.\PDF schedules'
if not os.path.exists(folder_location):os.mkdir(folder_location)

In [ ]:
url = "https://birminghamquality.org.uk/schedules/"

response = requests.get(url)
soup= BeautifulSoup(response.text, "html.parser")     
for link in soup.select("a[href$='.pdf']"):
    #Name the pdf files using the last portion of each link which are unique in this case
    filename = os.path.join(folder_location,link['href'].split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(requests.get(urljoin(url,link['href'])).content)

# Extract from PDF files

Need to delete a few non-schedule PDF files for this to work.

Extracts schedule for each scheme and saves as csv files

In [ ]:
import PyPDF2
import re
import pandas as pd

In [ ]:
ScheduleRegex = re.compile(r'''
                       (\d{1,4}) #Distribution
                       (\d\d\s\w\w\w\s\d\d) #Dispatch date
                       (\w\s[:]\s\d{1,2}) # Code
                       (\d\d\s\w\w\w\s\d\d) #Return date
                       ''',re.VERBOSE)

SchemeRegex = re.compile("for (.*) Birmingham Quality is part of the")
SchemeRegex2 = re.compile("Birmingham Quality (.*) Birmingham Quality is part")
df = pd.DataFrame()

for filename in os.listdir(folder_location):
    if not (filename.endswith('CALENDAR.pdf')):
        continue # skip non-PDF files

    pdfFileObj = open(os.path.join(folder_location,filename), 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    print('Opening...', filename)
    
    pageObj = pdfReader.getPage(0)
    text = pageObj.extractText()

    extracted_text = ScheduleRegex.findall(text)

    schedule = pd.DataFrame(extracted_text,columns=['Distribution','Dispatch date','Code','Return date'])
    try:
        schedule['Scheme'] = SchemeRegex.findall(text)[0]
    except:
        try:
            schedule['Scheme'] = SchemeRegex2.findall(text)[0]
        except:
            schedule['Scheme'] = 'Unknown'
    schedule = schedule.drop(['Code'],axis=1)
    
    df = pd.concat([df,schedule],ignore_index=True)
    
df.to_csv('schedule.csv', index=False)